### Tomographic Reconstruction (AreTomo)

In [ ]:
def get_aretomo_params():
    """Print out relevant AreTomo setup parameters"""
    aretomo_yaml = f"{os.getcwd()}/{proj_name}_aretomo_recon.yaml"
    if os.path.isfile(aretomo_yaml) is False:
        aretomo_yaml = f"{os.getcwd()}/{proj_name}_aretomo_align-recon.yaml"

    try:
        with open(aretomo_yaml, "r") as f:
            aretomo_params = yaml.load(f.read(), Loader=yaml.FullLoader)
            printmd(f"Vol Z = {aretomo_params['AreTomo_recon']['volz']}")
            printmd(
                f"Sample thickness = {aretomo_params['AreTomo_recon']['sample_thickness']}"
            )
            printmd(f"Pixel size = {aretomo_params['AreTomo_recon']['pixel_size']}")
            printmd(
                f"Reconstruction algorithm = {aretomo_params['AreTomo_recon']['recon_algo']}"
            )
            printmd(f"Bin factor = {aretomo_params['AreTomo_setup']['output_binning']}")
            printmd(f"Darktol = {aretomo_params['AreTomo_setup']['dark_tol']}")
    except FileNotFoundError:
        print(f"{aretomo_yaml} not found, cannot retrieve AreTomo setup parameters")


get_aretomo_params()

In [ ]:
def get_aretomo_tomofilenames():
    """Get list of tomograms from AreTomo"""
    aretomo_yaml = f"{os.getcwd()}/{proj_name}_aretomo_recon.yaml"
    if os.path.isfile(aretomo_yaml) is False:
        aretomo_yaml = f"{os.getcwd()}/{proj_name}_aretomo_align-recon.yaml"

    try:
        with open(aretomo_yaml, "r") as f:
            aretomo_params = yaml.load(f.read(), Loader=yaml.FullLoader)
            tomogram_filenames = aretomo_params["AreTomo_setup"]["output_mrc"]
            return tomogram_filenames
    except FileNotFoundError:
        print(f"{aretomo_yaml} not found, cannot retrieve tomogram filenames")


def get_central_slices(img_data):
    # Find central slices
    z_central = int(img_data.shape[0] / 2)
    y_central = int(img_data.shape[1] / 2)
    x_central = int(img_data.shape[2] / 2)

    # xy, yz, xz image data
    xy_data = img_data[z_central, :, :]
    yz_data = img_data[:, y_central, :]
    xz_data = img_data[:, :, x_central]

    return [xy_data, yz_data, xz_data]


def show_tomogram(tomo_fname):
    """tomo_fname can be 3-D .mrc, 3-D .tiff, or stack of 2-D tiffs"""

    # Import image data
    if tomo_fname.endswith(".mrc"):
        with mrcfile.open(tomo_fname) as mrc:
            img_data = mrc.data

    elif os.path.isdir(tomo_fname):
        volume = []
        for xy_slice in os.listdir(tomo_fname):
            volume.append(tifffile.imread(f"{tomo_fname}/{xy_slice}").data)
        img_data = np.stack(volume)

    elif tomo_fname.endswith(".tif"):
        img_data = tifffile.imread(tomo_fname)

    else:
        raise ValueError(
            "Tomogram cannot be read. Please use a 3D tiff, mrc or stack of 2D Tiffs"
        )

    yz_data, xy_data, xz_data = get_central_slices(img_data)
    # Show central slices in xy, yz, xz
    plt.figure(figsize=(10, 5))
    plt.suptitle(os.path.basename(tomo_fname))
    titles = ["x-y", "y-z", "x-z"]

    for i, data in enumerate([xy_data, yz_data, xz_data]):
        plt.subplot(1, 3, i + 1)
        plt.imshow(
            data,
            cmap="Greys_r",
            vmin=np.percentile(data.flatten(), 10),
            vmax=np.percentile(data.flatten(), 90),
        )
        plt.title(titles[i])

    plt.tight_layout()

def show_tomograms_as_grid(tomo_filenames: list):
    nrows = math.ceil(len(tomo_filenames) / 4)
    plt.figure(figsize=(10, nrows*3))

    for i, tomo in enumerate(tomo_filenames):
        plt.subplot(nrows, 4, i+1)
        with mrcfile.open(tomo) as mrc:
            plt.imshow(
                mrc.data, 
                cmap="Greys_r", 
                # vmin=np.percentile(mrc.data.flatten(), 10),
                # vmax=np.percentile(mrc.data.flatten(), 90),
            )
            plt.title(os.path.basename(tomo).strip("_rec_projXY.mrc"))
            plt.axis('off')

    plt.tight_layout()

In [ ]:
# Show thumbnails of the tomograms
tomogram_filenames = get_aretomo_tomofilenames()
xy_filenames = [f"{os.path.splitext(f)[0]}_projXY.mrc" for f in tomogram_filenames]
show_tomograms_as_grid(xy_filenames)

print("\n\n\n\n\n\n") # empty lines to avoid bottom being cut off in slides mode